#Stock Price Predictor (Using Linear Regression)

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import math, datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style

In [2]:
style.use('ggplot')

amd = yf.Ticker("AMD")
df = amd.history(period="max")

df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
df = df.mask(df['Open'].eq(0)).dropna()
df['HL_PCT'] = (df['High'] - df['Close']) / df['Close'] * 100.0
df['PCT_change'] = (df['Close'] - df['Open']) / df['Open'] * 100.0
df = df.mask(df['HL_PCT'].eq(0)).dropna()
df = df.mask(df['PCT_change'].eq(0)).dropna()

df = df[['Close', 'HL_PCT', 'PCT_change', 'Volume']]

print(df)

                               Close    HL_PCT  PCT_change      Volume
Date                                                                  
1983-03-21 00:00:00-05:00   9.000000  0.347222    0.348432    510800.0
1983-03-22 00:00:00-05:00   8.500000  5.514706   -2.508961    328000.0
1983-03-23 00:00:00-05:00   8.718750  1.075269    2.573529    427200.0
1983-03-25 00:00:00-05:00   8.875000  2.112676   -0.350877    210400.0
1983-03-28 00:00:00-05:00   8.687500  3.956835   -1.766784    494800.0
...                              ...       ...         ...         ...
2023-01-23 00:00:00-05:00  76.529999  0.078397    5.967872  84293200.0
2023-01-24 00:00:00-05:00  74.699997  1.258370    1.288131  60822600.0
2023-01-25 00:00:00-05:00  74.910004  0.280335    2.757205  49932600.0
2023-01-26 00:00:00-05:00  75.160004  2.554548   -1.751629  49583900.0
2023-01-27 00:00:00-05:00  75.400002  1.777183    2.306655  58066200.0

[8721 rows x 4 columns]


In [3]:
forecast_col = 'Close'
df.fillna(-99999, inplace=True)

forecast_out = int(math.ceil(0.001*len(df)))

df['label'] = df[forecast_col].shift(-forecast_out)

print(df)

                               Close    HL_PCT  PCT_change      Volume  \
Date                                                                     
1983-03-21 00:00:00-05:00   9.000000  0.347222    0.348432    510800.0   
1983-03-22 00:00:00-05:00   8.500000  5.514706   -2.508961    328000.0   
1983-03-23 00:00:00-05:00   8.718750  1.075269    2.573529    427200.0   
1983-03-25 00:00:00-05:00   8.875000  2.112676   -0.350877    210400.0   
1983-03-28 00:00:00-05:00   8.687500  3.956835   -1.766784    494800.0   
...                              ...       ...         ...         ...   
2023-01-23 00:00:00-05:00  76.529999  0.078397    5.967872  84293200.0   
2023-01-24 00:00:00-05:00  74.699997  1.258370    1.288131  60822600.0   
2023-01-25 00:00:00-05:00  74.910004  0.280335    2.757205  49932600.0   
2023-01-26 00:00:00-05:00  75.160004  2.554548   -1.751629  49583900.0   
2023-01-27 00:00:00-05:00  75.400002  1.777183    2.306655  58066200.0   

                             label  
